In [10]:
from unified_planning.io.pddl_reader import PDDLReader
from unified_planning.model import Problem
from unified_planning.shortcuts import *
from bitblast.utils import *
from bitblast.full_adder import *
from pathlib import Path
import os
import numpy as np
from unified_planning.io.pddl_writer import PDDLWriter


In [11]:
domain_path = Path(os.path.abspath('')).parent / "test" / "pddl" / "simple" / "domain.pddl"
problem_path = Path(os.path.abspath('')).parent / "test" / "pddl" / "simple" / "problem.pddl"

In [12]:
reader = PDDLReader()
problem = reader.parse_problem(domain_path, problem_path)

In [13]:
numeric_variables = get_numeric_variables(problem)
action_effect_map = get_action_effect_map(problem)
constants, init_constants = get_constants(problem, get_all_eff_num(problem))

fluent_to_var = {fluent: FluentExp(fluent) for fluent in numeric_variables}
var_to_fluent = {fluent_to_var[fluent]: fluent for fluent in numeric_variables}

constants_abs = {np.abs(q) for q in constants.union(init_constants)}
min_bits = int(np.ceil(np.log2(max(constants_abs) + 1)))

chosen_bits = 4
assert chosen_bits >= min_bits

bitblasted_variables, bitblasted_constants = get_bitblasted_variables(numeric_variables, constants, chosen_bits)
new_initial_values = get_bitblasted_initial_state(bitblasted_variables, bitblasted_constants, problem, chosen_bits)

In [14]:
bitblasted_variables

{real x: [bool x_0, bool x_1, bool x_2, bool x_3]}

In [15]:
action = problem.actions[0]
assert isinstance(action, InstantaneousAction)
new_action = InstantaneousAction(action.name)
eff = effects_num(action=action).pop()

var = var_to_fluent[eff.fluent]
value = constant_value(eff.value)

x_bits = bitblasted_variables[var]
q_bits = bitblasted_constants[value]
fa = FullAdder(x_bits, q_bits)

# Effect conversion
fa.circuit
sums = [fa.circuit[f"z_{i}"] for i in range(chosen_bits)]
condeffs = []

for i in range(chosen_bits):
    new_action.add_effect(condition=sums[i], fluent=FluentExp(x_bits[i]), value=TRUE())
    new_action.add_effect(condition=Not(sums[i]), fluent=FluentExp(x_bits[i]), value=FALSE())

new_action.effects

[if ((x_0 and (not q0_0)) or (q0_0 and (not x_0))) then x_0 := true,
 if (not ((x_0 and (not q0_0)) or (q0_0 and (not x_0)))) then x_0 := false,
 if ((((x_1 and (not q0_1)) or (q0_1 and (not x_1))) and (not (x_0 and q0_0))) or (x_0 and q0_0 and (not ((x_1 and (not q0_1)) or (q0_1 and (not x_1)))))) then x_1 := true,
 if (not ((((x_1 and (not q0_1)) or (q0_1 and (not x_1))) and (not (x_0 and q0_0))) or (x_0 and q0_0 and (not ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))))) then x_1 := false,
 if ((((x_2 and (not q0_2)) or (q0_2 and (not x_2))) and (not ((x_1 and q0_1) or (x_0 and q0_0 and ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))))) or (((x_1 and q0_1) or (x_0 and q0_0 and ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))) and (not ((x_2 and (not q0_2)) or (q0_2 and (not x_2)))))) then x_2 := true,
 if (not ((((x_2 and (not q0_2)) or (q0_2 and (not x_2))) and (not ((x_1 and q0_1) or (x_0 and q0_0 and ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))))) or (((x_1 and q0_1) or (x_0 an

In [16]:
new_problem = Problem(name='compiled')

for var, var_bits in bitblasted_variables.items():
    new_problem.add_fluents(var_bits)
    for var_bit in var_bits:
        initital_value = new_initial_values[var_bit]
        new_problem.set_initial_value(var_bit, initital_value)

for const, const_bits in bitblasted_constants.items():
    new_problem.add_fluents(const_bits)
    for const_bit in const_bits:
        initital_value = new_initial_values[const_bit]
        new_problem.set_initial_value(const_bit, initital_value)


new_problem.add_action(new_action)
new_problem

problem name = compiled

fluents = [
  bool x_0
  bool x_1
  bool x_2
  bool x_3
  bool q0_0
  bool q0_1
  bool q0_2
  bool q0_3
]

actions = [
  action increase_x {
    preconditions = [
    ]
    effects = [
      if ((x_0 and (not q0_0)) or (q0_0 and (not x_0))) then x_0 := true
      if (not ((x_0 and (not q0_0)) or (q0_0 and (not x_0)))) then x_0 := false
      if ((((x_1 and (not q0_1)) or (q0_1 and (not x_1))) and (not (x_0 and q0_0))) or (x_0 and q0_0 and (not ((x_1 and (not q0_1)) or (q0_1 and (not x_1)))))) then x_1 := true
      if (not ((((x_1 and (not q0_1)) or (q0_1 and (not x_1))) and (not (x_0 and q0_0))) or (x_0 and q0_0 and (not ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))))) then x_1 := false
      if ((((x_2 and (not q0_2)) or (q0_2 and (not x_2))) and (not ((x_1 and q0_1) or (x_0 and q0_0 and ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))))) or (((x_1 and q0_1) or (x_0 and q0_0 and ((x_1 and (not q0_1)) or (q0_1 and (not x_1))))) and (not ((x_2 and (not q0_2)) 

In [17]:
writer = PDDLWriter(new_problem)
new_domain_path = Path(os.path.abspath('')).parent / "test" / "pddl" / "simple" / "domain_compiled.pddl"
new_problem_path = Path(os.path.abspath('')).parent / "test" / "pddl" / "simple" / "problem_compiled.pddl"
writer.write_domain(new_domain_path)
writer.write_problem(new_problem_path)